# Создание решения по стандартизации названий спортивных школ.

**Описание проекта**

<br>Сервис "Мой Чемпион" помогает спортивным школам фигурного катания, тренерам
<br>мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.

**Цель**

- Создать решение для стандартизации названий спортивных школ.
  <br>Например, одна и та же школа может быть записана по-разному
  <br>Необхдодимо сопоставить эти варианты эталонному названию из предоставленной таблицы

**Задачи**

- Изучить данные – эталонные названия СШ и варианты пользовательского ввода
- Подготовить обучающий набор данных на основе эталонного датасета
- Создать модель для подбора наиболее вероятных названий при ошибочном вводе
- Создать функцию (класс, модуль) для применения в сервисе
  - возможность выбора количества кандидатов
  - вывод в виде списка словарей
- Протестировать решение
- Проанализировать результат и предложить варианты улучшения
- Создать документацию
  - описание признаков
  - какая модель используется
  - как оценивается качество
  - инструкция по запуску (применению)
- Создать демо приложение

**План выполнения**

- Загрузка данных;
- Объединение наборов данных;
- Исследовательский анализ данных;
- Чистка данных;
- Получение эмбеддингов;
- Обучение модели;
- Проверка работоспособности;
- Заключение.

## Установка зависимостей

In [13]:
# !pip install -U sentence-transformers
# !pip install pandas

!pip freeze > requirements.txt

In [14]:
import os
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

Объявление глобальных переменных

In [2]:
DATA_DIR = 'data'
MODEL_DIR = 'services/models'
RANDOM_STATE = 42
TEST_SIZE = 0.1

pd.options.mode.copy_on_write = True

## Знакомство с данными

In [3]:
schools = pd.read_csv(os.path.join(DATA_DIR, 'Школы.csv'))
schools.tail()

,school_id,name,region
301,305,Прогресс,Алтайский край
302,609,"""СШ ""Гвоздика""",Удмуртская республика
303,610,"СШОР ""Надежда Губернии",Саратовская область
304,611,КФК «Айсберг»,Пермский край
305,1836,"ООО ""Триумф""",Москва


In [4]:
sample = pd.read_csv(os.path.join(DATA_DIR, 'Примерное написание.csv'))
sample.head()

,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""


Выводы и наблюдения:
- оба датасета загружены;
- наблюдается общий признак для объединения;
- одной записи в эталонном датасете может соответствовать несколько записей во втором.

## Объединение датасетов

In [5]:
df = sample.merge(schools, how='left', on='school_id')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  895 non-null    int64 
 1   name_x     895 non-null    object
 2   name_y     895 non-null    object
 3   region     895 non-null    object
dtypes: int64(1), object(3)
memory usage: 28.1+ KB


In [6]:
df.rename(columns={'name_x': 'name', 'name_y': 'target'}, inplace=True)
df['standard'] = df['target'] + ': ' + df['region']
df.drop(columns=['target', 'region'], axis=1, inplace=True)
df.tail(10)

,school_id,name,standard
885,5,Айсдрим,Ice Dream / Айс Дрим: Санкт-Петербург
886,4,КФК Аврора,Аврора: Санкт-Петербург
887,4,"МО г.Петергоф, СФК «Аврора»",Аврора: Санкт-Петербург
888,3,"Республика Татарстан, МБУ СШОР ""ФСО ""Авиатор""",Авиатор: Республика Татарстан
889,3,"Республика Татарстан, МБУ СШОР ""ФСО ""Авиатор""""...",Авиатор: Республика Татарстан
890,3,"Республика Татарстан, СШОР ФСО Авиатор",Авиатор: Республика Татарстан
891,3,"СШОР ФСО Авиатор, Республика Татарстан",Авиатор: Республика Татарстан
892,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»",Авиатор: Республика Татарстан
893,2,"ЯНАО, СШ ""Авангард""",Авангард: Ямало-Ненецкий АО
894,1,"Московская область, СШ ""Авангард""",Авангард: Московская область


Выводы и наблюдения:
- эталонный набор данных присоединён ко второму;
- произведено объединение названия школы с регионом.

## Исследовательский анализ данных

In [7]:
print(f"Количество полных дубликатов строк: {df.duplicated().sum()}")
print(f"Количество уникальных значений целевой переменной: {df['standard'].nunique()}")
print(f"Количество дубликатов строк по признаку 'name': {df['name'].duplicated().sum()}")
print('Совпадения строк в признаке name:')
display(df[df.duplicated(subset='name', keep=False)])
df = df[df['school_id'] != 277]
print('Удалена строка с повторяющимся name, но разным standard')
df.info()

Количество полных дубликатов строк: 0
Количество уникальных значений целевой переменной: 264
Количество дубликатов строк по признаку 'name': 1
Совпадения строк в признаке name:


,school_id,name,standard
34,277,"КФК ""Динамо-Санкт-Петербург""","НП КФК ""Динамо-Санкт-Петербург"": Санкт-Петербург"
758,48,"КФК ""Динамо-Санкт-Петербург""",Динамо Санкт-Петербург: Санкт-Петербург


Удалена строка с повторяющимся name, но разным standard
<class 'pandas.core.frame.DataFrame'>
Index: 893 entries, 0 to 894
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  893 non-null    int64 
 1   name       893 non-null    object
 2   standard   893 non-null    object
dtypes: int64(1), object(2)
memory usage: 27.9+ KB


In [8]:
print(f"В среднем на одно корректное наименование приходится {round(df['name'].nunique() / df['standard'].nunique(),)} некорректных")

В среднем на одно корректное наименование приходится 3 некорректных


Выводы и наблюдения:
- полных повторов в данных нет;
- пропусков в данных нет;
- удалена одна строка с повторяющимся значением в признаке `name`.

## Получение эмбеддингов

Загрузка модели

In [9]:
try:
    labse = joblib.load(os.path.join(MODEL_DIR, 'labse.model'))
except:
    labse = SentenceTransformer('sentence-transformers/LaBSE')
    joblib.dump(labse, os.path.join(MODEL_DIR, 'labse.model'))
labse

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 768, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
  (3): Normalize()
)

Получение эмбеддингов наименований школ

In [15]:
school_names = df['name'].values

# Wrap the list of school names with tqdm for progress bar
with tqdm(school_names) as progress_bar:
  name_embeddings = []
  for name in progress_bar:
    embedding = labse.encode(name)
    name_embeddings.append(embedding)

name_embeddings = np.array(name_embeddings)  # Convert list to numpy array
name_embeddings.shape

100%|██████████████████████████████████████████████████████████████████| 893/893 [02:07<00:00,  6.99it/s]


(893, 768)

In [16]:
df.sample(20)

,school_id,name,standard
430,144,"Краснодарский край, РЦСП по ФКК",РЦСП по ФФК: Краснодарский край
644,82,"Москва, ГБУ ДО ""МАФК, Школа ""Легенда""",Легенда: Москва
502,119,"г. Москва, ГБУ ДО ""МАФКК"" Олимп",Олимп: Москва
143,240,"СШ Ямал, ЯНАО",ФФКК ЯНАО: Ямало-Ненецкий АО
691,62,СПб ГБУ СШОР Звездный лед,Звездный лед: Санкт-Петербург
795,37,"Нижегородская обл., ГБУ НО СШОР по ЛСВС,г.Нижн...",ГБОУ ДО НО СШОР по ЛВС: Нижегородская область
232,206,"СШОР №1, Самарская область",СШОР No 1: Самарская область
808,33,"Самарская область, СШОР №13 ""Волгарь""",Волгарь: Самарская область
880,7,"Рязанская область, ГАУ ДО РО ""СШ ""Айсберг""",Айсберг: Рязанская область
522,111,ЧОУ ДОД ДЮСШ Невский лед,Невский лед: Санкт-Петербург


In [54]:
input_school_name = 'Космос'

In [56]:
import sentence_transformers
res = sentence_transformers.util.semantic_search(labse.encode(input_school_name), name_embeddings, top_k=5)

idx = [i['corpus_id'] for i in res[0]]
score = [i['score'] for i in res[0]]
result = (df.loc[idx, ['school_id', 'name', 'standard']]
          .assign(cosine_similarity=score)
          .drop_duplicates(subset=['school_id', 'name'])
          .iloc[:5])
result

,school_id,name,standard,cosine_similarity
873,10,"АНО АСКК, Москва",Академия синхронного катания на коньках: Москва,0.489191
658,79,ХШФК Красная звезда,Красная звезда: Санкт-Петербург,0.474319
660,78,"Калужская область, СШ ""Космос""",Космос: Калужская область,0.454622
778,41,"Тверская область, МБОУ ДО ДЮСШ им. И.Я. Гусева...",Гусева: Тверская область,0.437746
396,161,"Москва, МАФКК школа ""Сокольники""",Сокольники: Москва,0.402909


In [25]:
result

,school_id,name,standard,cosine_similarity
373,168,"Пермский край, СШОР ""Старт""",Старт: Пермский край,1.000000
374,168,"Пермский край, ГБУ ДО ПК «СШОР «Старт»",Старт: Пермский край,0.998756
366,169,"Свердловская область, ФКиС Старт",Старт: Свердловская область,0.948739
